In [1]:
using DataFrames, CSV, Impute, Dates, Plots, Statistics, Interpolations

In [2]:
include("utils.jl");

## Choose data file

In [3]:
choice = "boston_18_20";

In [4]:
df = CSV.File(string("data/", choice, ".csv")) |> DataFrame;

In [5]:
dfdaily = filter(row -> strip(row[:REPORT_TYPE]) == "SOD", df);
dfh = filter(row -> !(strip(row[:REPORT_TYPE]) in ["SOD", "SOM"]), df)
sum(ismissing.(dfh[:, :HourlyDryBulbTemperature]))

0

In [8]:
factors =  [(:HourlyDryBulbTemperature,:temp)];

In [9]:
mdata = selectdata(dfh, factors);
first(mdata, 4)

,sampleT,temp
,DateTime,Float64
1,2018-01-01T01:00:00,2.0
2,2018-01-01T02:00:00,1.0
3,2018-01-01T03:00:00,1.0
4,2018-01-01T04:00:00,1.0


## Define characteristic of training set

In [12]:
start = 2018, 1, 2
dayslookback = 7
ndays = 360
predicttype = :temp
offset = 1;

In [13]:
X, y = build_multi_data(mdata, predicttype, start, dayslookback, ndays; offset=offset);

In [14]:
(train_X, train_y), (test_X, test_y) = IAI.split_data(:regression, X, y);

In [15]:
@time begin
grid = IAI.GridSearch(
    IAI.OptimalTreeRegressor(
        random_seed=123,
        show_progress=false
    ),
    max_depth = 5:10,
    minbucket = 10, #10:10:100
    show_progress=false  
)
IAI.fit!(grid, train_X, train_y)
end

┌ Warning: This copy of Interpretable AI software is for academic purposes only and not for commercial use.
└ @ IAILicensing C:\Users\iai\builds\InterpretableAI\SysImgBuilder\.julia\packages\IAILicensing\x1CT6\src\precompile.jl:19
┌ Warning: Interpretable AI license expires soon: 2020-12-31. If you need to renew, please send us the following machine ID:
│ 1d23b3eee464c21ff412aa6a5b247b9f0543ded03e05d39d863fbfc9241e7a1f
└ @ IAILicensing C:\Users\iai\builds\InterpretableAI\SysImgBuilder\.julia\packages\IAILicensing\x1CT6\src\precompile.jl:29


4250.509310 seconds (32.81 M allocations: 2.335 GiB, 0.09% gc time)


All Grid Results:

│ Row │ show_progress │ minbucket │ max_depth │ cp         │ train_score │
│     │ Bool          │ Int64     │ Int64     │ Float64    │ Float64     │
├─────┼───────────────┼───────────┼───────────┼────────────┼─────────────┤
│ 1   │ false         │ 10        │ 5         │ 1.41321e-5 │ 0.989181    │
│ 2   │ false         │ 10        │ 6         │ 9.14544e-6 │ 0.990902    │
│ 3   │ false         │ 10        │ 7         │ 3.02208e-5 │ 0.991885    │
│ 4   │ false         │ 10        │ 8         │ 1.83949e-5 │ 0.992795    │
│ 5   │ false         │ 10        │ 9         │ 1.69718e-5 │ 0.992271    │
│ 6   │ false         │ 10        │ 10        │ 1.66926e-5 │ 0.992398    │

│ Row │ valid_score │ rank_valid_score │
│     │ Float64     │ Int64            │
├─────┼─────────────┼──────────────────┤
│ 1   │ 0.988076    │ 2                │
│ 2   │ 0.988428    │ 1                │
│ 3   │ 0.987299    │ 3                │
│ 4   │ 0.986626    │ 4                │
│ 5   │ 0.98356   

In [17]:
lnr = IAI.get_learner(grid)
IAI.score(lnr, test_X, test_y)  # Testing on the same year

0.9897658118974852

In [16]:
start = 2019, 1, 1
dayslookback = 7
ndays = 365-dayslookback-1
outsample_X, outsample_y = build_multi_data(mdata, predicttype, start, dayslookback, ndays; offset=offset);